## Quadrotor Obstacle Avoidance with Visualization

In [1]:
using TrajectoryOptimization
using Plots, LinearAlgebra

┌ Info: Recompiling stale cache file /home/taylor/.julia/compiled/v1.1/TrajectoryOptimization/UVgeA.ji for TrajectoryOptimization [c79d492b-0548-5874-b488-5a62c1d9d0ca]
└ @ Base loading.jl:1184
┌ Warning: Package TrajectoryOptimization does not have MatrixCalculus in its dependencies:
│ - If you have TrajectoryOptimization checked out for development and have
│   added MatrixCalculus as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with TrajectoryOptimization
└ Loading MatrixCalculus into TrajectoryOptimization from project dependency, future warnings for TrajectoryOptimization are suppressed.


Import the quadrotor model

In [37]:
model = Dynamics.quadrotor_model
n = model.n # number of states
m = model.m; # number of controls

Define numerical type

In [38]:
T = Float64;

Define initial and goals states

In [72]:
q0 = [1.;0.;0.;0.] # unit quaternion

x0 = zeros(T,n)
x0[1:3] = [0.; 0.; 1.]
x0[4:7] = q0

xf = zero(x0)
xf[1:3] = [0.;40.; 1.]
xf[4:7] = q0;

Define a cost function, e.g., quadratic

In [73]:
Q = (1.0e-2)*Diagonal(I,n)
R = (1.0e-1)*Diagonal(I,m)
Qf = 1000.0*Diagonal(I,n)
cost = LQRCost(Q, R, Qf, xf);

Define constraints

In [74]:
bnd = bound_constraint(n,m,u_min=0.0,u_max=5.0,trim=true); # stagewise constraint
goal = goal_constraint(xf)

# create obstacle constraints
r_quad = 1.0
r_sphere = 2.0
spheres = ((0.,10.,0.,r_sphere),(0.,20.,0.,r_sphere),(0.,30.,0.,r_sphere))
n_spheres = 3

function sphere_obs3(c,x,u)
    for i = 1:n_spheres
        c[i] = TrajectoryOptimization.sphere_constraint(x,spheres[i][1],spheres[i][2],spheres[i][3],spheres[i][4]+r_quad)
    end
    return nothing
end

obs = Constraint{Inequality}(sphere_obs3,n,m,n_spheres,:obs)

con = [bnd,obs,goal]; # constraint set

Define a problem

In [75]:
N = 101 # number of knot points
dt = 0.1 # total time

U = [0.5*9.81/4.0*ones(m) for k = 1:N-1] # initial hovering control trajectory
obj = TrajectoryOptimization.Objective(cost,N) # objective with same stagewise costs

con = TrajectoryOptimization.ProblemConstraints(con,N) # constraint trajectory

prob = Problem(model,obj, constraints=con, x0=x0, integration=:rk4, N=N, dt=dt)
initial_controls!(prob,U); # initialize problem with controls

Solve problem

In [76]:
solve!(prob, TrajectoryOptimization.ALTROSolverOptions{T}(verbose=true)) # solve with iLQR

Visualization

In [85]:
using MeshCat, GeometryTypes, CoordinateTransformations, FileIO, MeshIO

In [86]:
vis = Visualizer()
open(vis)

┌ Info: Serving MeshCat visualizer at http://127.0.0.1:8705
└ @ MeshCat /home/taylor/.julia/packages/MeshCat/WlA0B/src/servers.jl:24


Process(`xdg-open http://127.0.0.1:8705`, ProcessExited(0))

In [87]:
# color options
red_ = MeshPhongMaterial(color=RGBA(1, 0, 0, 1.0))
black_ = MeshPhongMaterial(color=RGBA(0, 0, 0, 1.0))
blue_ = MeshPhongMaterial(color=RGBA(0, 0, 1, 1.0));

In [88]:
# Import quadrotor obj file
traj_folder = joinpath(dirname(pathof(TrajectoryOptimization)),"..")
urdf_folder = joinpath(traj_folder, "dynamics","urdf")
obj = joinpath(urdf_folder, "quadrotor_base.obj")
robot_obj = FileIO.load(obj)

robot = vis["robot"]
robot_obj.vertices .= robot_obj.vertices .* 0.33
setobject!(vis["robot"]["quad"],robot_obj,black_);

In [89]:
# geometries
sphere_small = HyperSphere(Point3f0(0), convert(Float32,0.1*r_quad)) # trajectory points
sphere_medium = HyperSphere(Point3f0(0), convert(Float32,r_quad));

In [90]:
# Create and place obstacles
obstacles = vis["obs"]

for i = 1:n_spheres
    setobject!(vis["obs"]["s$i"],HyperSphere(Point3f0(0), convert(Float32,spheres[i][4])),red_)
    settransform!(vis["obs"]["s$i"], Translation(spheres[i][1], spheres[i][2], spheres[i][3]))
end

In [91]:
# Create and place trajectory
traj = vis["traj"]

for i = 1:N
    setobject!(vis["traj"]["t$i"],sphere_small,blue_)
    settransform!(vis["traj"]["t$i"], Translation(prob.X[i][1], prob.X[i][2], prob.X[i][3]))
end

In [92]:
# Set camera location
settransform!(vis["/Cameras/default"], compose(Translation(25., 15., 20.),LinearMap(RotY(-pi/12))));

In [93]:
# Animation
anim = MeshCat.Animation(20)
for i = 1:N
    MeshCat.atframe(anim,vis,i) do frame
        settransform!(frame["robot"], compose(Translation(prob.X[i][1:3]...),LinearMap(Quat(prob.X[i][4:7]...))))
    end
end
MeshCat.setanimation!(vis,anim)